In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr

from segysak import open_seisnc

from src.definitions import ROOT_DIR
from src.data import make_sua_surfaces
from src.visualization.visualize import save_seismic_basemap

# Load seismic

In [ ]:
# Downloaded files directory
dst_dir = ROOT_DIR / "data"
seisnc_path = dst_dir / "interim/R3136_15UnrPrDMkD_Full_D_Rzn_RMO_Shp_vG.seisnc"

In [ ]:
seisnc = open_seisnc(seisnc_path, chunks={"inline": 100})

# Load regional surfaces

In [ ]:
surface_names = ["ns_b", "ck_b", "rnro1_t", "ze_t", "ro_t"]

In [ ]:
surfaces = {
    surface_name: make_sua_surfaces.load_mapped_horizon(surface_name)
    for surface_name in surface_names
}

# Load summary surfaces

In [ ]:
dst_dir = ROOT_DIR / "data/processed/summary"
dst = dst_dir / "quantiles.nc"

In [ ]:
quantiles = xr.open_dataarray(dst)

# Plot surfaces

# Figure 0: Basemap with inline

In [ ]:
save_seismic_basemap()

## Figure 1: Inline with regional surfaces and quantiles.

In [ ]:
inl_sel = 9100
xline_range = slice(7570, 9600)
depth_slice = slice(0, 4000)

In [ ]:
opt = dict(
    x="xline",
    y="depth",
    add_colorbar=True,
    interpolation="spline16",
    robust=True,
    yincrease=False,
    cmap="Greys",
)

f, ax = plt.subplots(figsize=(16, 10), constrained_layout=True)

seisnc.data.sel(
    iline=inl_sel,
    xline=xline_range,
    depth=depth_slice,
    ).plot.imshow(ax=ax, **opt)

for surface_name, surface in surfaces.items():
    trace = surface.sel(iline=inl_sel, xline=xline_range)
    ax.plot(trace.xline, trace.values, label=surface_name)

for q in quantiles["quantile"]:
    q = q.values
    quantile_trace = quantiles.sel(
        iline=inl_sel,
        xline=xline_range,
        quantile=q
    )
    ax.plot(
        quantile_trace.xline,
        quantile_trace.values,
        label=f"RO T P{q*100:.0f}"
    )

ax.invert_xaxis()

## Figure 2: Inline zoomed in with Top of Salt and RO

In [ ]:
from src.visualization.visualize import save_fig_inline_with_regional_and_summary_surfaces

In [ ]:
inl_sel = 9100
xline_min = 7570
xline_max = 7900
depth_min = 0
depth_max = 3500

In [ ]:
save_fig_inline_with_regional_and_summary_surfaces(
    filename="Zoomed_inline_9100.png",
    inl_sel=inl_sel,
    xline_min=xline_min,
    xline_max=xline_max,
    depth_min=depth_min,
    depth_max=depth_max,
    regional_surfaces=["ns_b", "ck_b", "rnro1_t"],
    quantile_surfaces=[],
)

## Figure 3: Inline with all SUA proxy surfaces

In [ ]:
from src.visualization.visualize import save_fig_inline_with_proxy_surfaces

In [ ]:
inl_sel = 9100
xline_min = 7625
xline_max = 7900
depth_min = 2700
depth_max = 3200

In [ ]:
save_fig_inline_with_proxy_surfaces(
    inl_sel=inl_sel,
    xline_min=xline_min,
    xline_max=xline_max,
    depth_min=depth_min,
    depth_max=depth_max,
)

# Figure 4: Inline with summary surfaces

In [ ]:
inl_sel = 9100
xline_min = 7625
xline_max = 7900
depth_min = 2700
depth_max = 3200

In [ ]:
save_fig_inline_with_regional_and_summary_surfaces(
    filename=f"Inline_{inl_sel}_with_summary_surfaces.png",
    inl_sel=inl_sel,
    xline_min=xline_min,
    xline_max=xline_max,
    depth_min=depth_min,
    depth_max=depth_max,
    regional_surfaces=[],
    quantile_surfaces=[0.10, 0.25, 0.50, 0.75, 0.90],
    show_legend=True,
)